In [ ]:
import requests as req
import pandas as pd
import copy

In [ ]:
req_url = 'https://einfo.ceproas.cz/cepro_portal_ws/rest/common/prox/mobileData'
req_header = {'authorization': 'Basic bW9iYXA6RVdpa0Ey',
              'content-type': 'application/json; charset=UTF-8',
              'accept-encoding': 'gzip',
              'user-agent': 'okhttp/4.9.0'}
json_body_encyclopedia = '{"encyklopedie": {"full": true}, "version": 5}'
json_body_stations = '{"search": {"exclude_cs_ceny": false,"exclude_cs_kvalita": false},"version": 5}'

In [ ]:
try:
    response_gas_stations = req.post(url=req_url, headers=req_header, data=json_body_stations, timeout = 10, verify=False)
    response_encyclopedia = req.post(url=req_url, headers=req_header, data=json_body_encyclopedia, timeout = 10, verify=False)
    response_gas_stations.raise_for_status()
    response_encyclopedia.raise_for_status()
except req.exceptions.Timeout as errrt:
    print("ERROR : ceproas.cz API - Request Time out. CEPRO server is down.")
except req.exceptions.RequestException:
    print('ERROR : Something went wrong.')

In [ ]:
response_gas_stations = req.post(url=req_url, headers=req_header, data=json_body_stations, timeout = 10)

In [ ]:
# pd.to_pickle(response_gas_stations, '_data/response_gas_stations.pickle')
# pd.to_pickle(response_encyclopedia, '_data/response_encyclopedia.pickle')

response_gas_stations = pd.read_pickle('_data/stations.pickle')
response_encyclopedia = pd.read_pickle('_data/encyclopedia.pickle')

In [ ]:
# for development
all_gs = copy.deepcopy(response_gas_stations.json())
encyclopedia_info = copy.deepcopy(response_encyclopedia.json())

# check for errors in json files
if all_gs['success'] is not True:
    error_msg_all_gs = all_gs['Error']['errorText']
    print(error_msg_all_gs)

if encyclopedia_info['success'] is not True:
    error_msg_encyclopedia_info = encyclopedia_info['Error']['errorText']
    print(error_msg_encyclopedia_info)

# gas stations
data_gs = all_gs['Data']

# product list
products_list = pd.DataFrame(data_gs['cis_prod_list'])

# fuel prices
gs_prices = pd.concat([
    pd.DataFrame(data_gs['cs_ceny']).explode('ceny').reset_index()
    , pd.DataFrame(pd.DataFrame(data_gs['cs_ceny']).explode('ceny')['ceny'].tolist()).reset_index()
    ]
    , axis=1)[['kod_cs', 'cena', 'kod_produkt']]
gs_prices = gs_prices.merge(products_list, how='inner', on='kod_produkt')



# Product quality list
product_quality_list = pd.DataFrame(data_gs['cis_kvalita_list'])

# product quality
gs_product_quality = pd.DataFrame()
for gs_product in data_gs['cs_kvalita']:
    gs_number = gs_product['kod_cs']
    product_quality_gs = pd.DataFrame(pd.DataFrame(gs_product['produkty']).explode('parametry')['parametry'].to_list())
    product_quality_gs['kod_cs'] = gs_number
    gs_product_quality = pd.concat([gs_product_quality,product_quality_gs], axis='index')
gs_product_quality = gs_product_quality.merge(product_quality_list, how='inner', on='kod_parametr')

# gas station location
gs_adress_wo_gps = pd.DataFrame(data_gs['cs_fix_list']).reset_index()
gs_adress = pd.concat([gs_adress_wo_gps, pd.DataFrame(gs_adress_wo_gps['GPS'].tolist())], axis=1)
del(gs_adress_wo_gps)




# encyclopedia
# info = encyclopedia_info['Encyklopedie']
# for i in info['tree']:
#     if i['parent'] == 'root':
#         root = pd.DataFrame(i['children'])
#     if i['parent'] == 'produkt':
#         product = pd.DataFrame(i['children'])
#     if i['parent'] == 'kvalita':
#         quality = pd.DataFrame(i['children'])
#     if i['parent'] == 'kvalita:natural95':
#         quality_95 = pd.DataFrame(i['children'])
#     if i['parent'] == 'kvalita:optimal_diesel':
#         quality_diesel = pd.DataFrame(i['children'])

In [ ]:
gs_prices # cena paliva
# gs_product_quality # hodnoty kvality pro kazdou benzinu
# product_quality_list  # list parmaetru kvality, uz nepotrebuju
# products_list # produkt ciselnik, uz nepotrebuju
gs_adress # adresa,kraje GPS


In [ ]:
from cepro_fuel.utils import data_download


In [ ]:
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)
import pandas as pd
import streamlit as st
from enum import Enum, unique
import requests as req

@unique
class info_type_enum(Enum):
    encyclopedia = 'encyclopedia'
    stations = 'stations'

@unique
class request_body_enum(Enum):
    encyclopedia = '{"encyklopedie": {"full": true}, "version": 5}'
    stations = '{"search": {"exclude_cs_ceny": false,"exclude_cs_kvalita": false},"version": 5}'
def data_download(info_type: info_type_enum, dev_mode: bool | None):
    req_url = 'https://einfo.ceproas.cz/cepro_portal_ws/rest/common/prox/mobileData'
    req_header = {'authorization': 'Basic bW9iYXA6RVdpa0Ey',
              'content-type': 'application/json; charset=UTF-8',
              'accept-encoding': 'gzip',
              'user-agent': 'okhttp/4.9.0'}
    if dev_mode == True:
        # For development, load files from project
        response = pd.read_pickle(f'_data/{info_type.value}.pickle')
        return response
    else:
        try:
            response = req.post(url=req_url, headers=req_header, data=request_body_enum[info_type.value].value, timeout = 10)
            response.raise_for_status()
            return response.json()
        except req.exceptions.Timeout as errrt:
            st.error("ERROR : ceproas.cz API - Request Time out. CEPRO server is down.")
            st.stop()
        except req.exceptions.RequestException:
            st.error('ERROR : Something went wrong.')
            st.stop()
def check_for_errors_in_response(response_json):
    #  check for errors in json files
    if response_json['success'] is not True:
        st.error(response_json['Error']['errorText'])

In [ ]:
from decimal import Decimal
from typing import List
class fuel_item():
    """Describes a single item in terms of price and product type"""
    price: Decimal
    product: int

class station_location():
    county: str
    town: str
    latitude: Decimal
    longitude: Decimal

class gas_station():
    """Holds info for various areas such as, location, products, etc"""
    fuel_items: List[fuel_item]
    station_id: int
    station_adress: station_location


In [ ]:
response = data_download(info_type = info_type_enum.stations, dev_mode=True)
response

In [ ]:
data_gs = all_gs['Data']

# product list
products_list = pd.DataFrame(data_gs['cis_prod_list'])

In [ ]:
import json
# json.load(data_gs['cis_prod_list'])
simple_dictionary = dict(data_gs['cis_prod_list'])
simple_dictionary = {i for i in data_gs['cis_prod_list']}

In [32]:
products_list = pd.DataFrame(data_gs['cis_prod_list'])[['nazev_produkt', 'kod_produkt']]

In [37]:
stations_data = all_gs['Data']

products_list = pd.DataFrame(stations_data['cis_prod_list'])[['nazev_produkt', 'kod_produkt']]
gs_prices = pd.concat([
    pd.DataFrame(stations_data['cs_ceny']).explode('ceny').reset_index()
    , pd.DataFrame(pd.DataFrame(stations_data['cs_ceny']).explode('ceny')['ceny'].tolist()).reset_index()
    ]
    , axis=1)[['kod_cs', 'cena', 'kod_produkt']]
gs_prices = gs_prices.merge(products_list, how='inner', on='kod_produkt')

In [39]:
product_quality_list = pd.DataFrame(stations_data['cis_kvalita_list'])[['jednotka_parametr', 'kod_parametr', 'nazev_parametr']]
gs_product_quality = pd.DataFrame()
for gs_product in stations_data['cs_kvalita']:
    gs_number = gs_product['kod_cs']
    product_quality_gs = pd.DataFrame(pd.DataFrame(gs_product['produkty']).explode('parametry')['parametry'].to_list())
    product_quality_gs['kod_cs'] = gs_number
    gs_product_quality = pd.concat([gs_product_quality,product_quality_gs], axis='index')
gs_product_quality = gs_product_quality.merge(product_quality_list, how='inner', on='kod_parametr')

In [40]:
gs_product_quality

,kod_parametr,hodnota,kod_cs,jednotka_parametr,nazev_parametr
0,1-1,828.7,21,<div>kg/m&sup3;</div>,Hustota
1,1-1,831.3,22,<div>kg/m&sup3;</div>,Hustota
2,1-1,830.1,31,<div>kg/m&sup3;</div>,Hustota
3,1-1,829.3,41,<div>kg/m&sup3;</div>,Hustota
4,1-1,837.5,51,<div>kg/m&sup3;</div>,Hustota
...,...,...,...,...,...
1243,4-3,204.8,620,<div>&deg;C</div>,Konec destilace
1244,4-3,208.8,621,<div>&deg;C</div>,Konec destilace
1245,4-3,208.8,622,<div>&deg;C</div>,Konec destilace
1246,4-3,206.3,624,<div>&deg;C</div>,Konec destilace
